In [1]:
import json, shutil
import os, re, sys
from pathlib import Path
import glob, time


import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

import numpy as np

import pandas as pd



from tqdm import tqdm
from tqdm import notebook
notebook.tqdm.pandas()
tqdm.pandas()

import psutil
from pyxtension.streams import stream


import matplotlib.pyplot as plt
plt.rcParams["figure.figsize"] = (20, 12)


import preprocessers as r
import pt_model as m

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\yazee\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to
[nltk_data]     C:\Users\yazee\AppData\Roaming\nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


In [2]:
LOAD_NUM = 100
RANDOM_SEED = 42
PROCESSORS_COUNT = psutil.cpu_count(logical=False)
MULTI = PROCESSORS_COUNT * int(str(LOAD_NUM) [:-2])

np.random.seed(RANDOM_SEED)

data_dir = Path('.')
train_dir = os.path.join(data_dir, 'train')

models_dir = os.path.join(data_dir, 'pt_models')
orders_path = os.path.join(data_dir, 'train_orders.csv')
ancestors_path = os.path.join(data_dir, 'train_ancestors.csv')

# shutil.rmtree(models_dir)
# if not os.path.exists(models_dir):
#     os.mkdir(models_dir)

print(f"\033[94mNumber of notebooks present in train set  = ", len(list(glob.iglob(os.path.join(train_dir, '*.json')))))

Number of notebooks present in train set  =  139256


In [3]:
start_time = time.time()
df = r.read_all_notebooks_(train_dir, LOAD_NUM, PROCESSORS_COUNT)

df = pd.concat(stream(np.array_split(df, PROCESSORS_COUNT)).mpmap(r.extract_features))

display(df.head())

print('-' * 125)

# Read Ordering data
df_orders = pd.read_csv(
    orders_path,
    index_col='id',
)
df_orders['cell_order'] = df_orders['cell_order'].str.split()  # Split the string representation of cell_ids into a list
df_orders = df_orders.squeeze(axis=1)


# build ranks as integers 
df = df.join(r.build_ranks_(df_orders, df, PROCESSORS_COUNT))


# Read Ancestors data
df = df.reset_index().merge(pd.read_csv(ancestors_path,  index_col='id'), on=["id"])

# convert integer ranks to percentages 
df["rank"] = df["rank"] / df.groupby("id")["cell_id"].transform("count")

display(df.head())

rduration = time.time() - start_time
print(f"Duration {rduration} seconds")

Train NBs: 100%|██████████| 100/100 [00:00<?, ?it/s]


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 4427 entries, ('00001756c60be8', '1862f0a6') to ('002bcc9e2f9077', 'cffa684c')
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   cell_type  4427 non-null   category
 1   source     4427 non-null   object  
dtypes: category(1), object(1)
memory usage: 220.6+ KB
None


cell_type  \
id             cell_id              
00001756c60be8 1862f0a6      code   
               2a9e43d6      code   
               038b763d      code   
               2eefe0ef      code   
               0beab1cd      code   

                                                                    source  \
id             cell_id                                                       
00001756c60be8 1862f0a6  #  This Python 3 environment comes with many h...   
               2a9e43d6  import numpy as np\nimport pandas as pd\nimpor...   
               038b763d  import warnings\nwarnings.filterwarnings('igno...   
               2eefe0ef      matplotlib.rcParams.update({'font.size': 14})   
               0beab1cd  def evaluate_preds(train_true_values, train_pr...   

                         n_code_cells  n_markdown_cells  words_count  \
id             cell_id                                                 
00001756c60be8 1862f0a6      0.517241          0.482759          140   
               2a9e43d6      0.517241          0.482759           55   
               038b763d      0.517241          0.482759            3   
               2eefe0ef      0.517241          0.482759            2   
               0beab1cd      0.517241          0.482759           39   

                         letters_count  empty_lines_count  \
id             cell_id                                      
00001756c60be8 1862f0a6            930           0.235294   
               2a9e43d6            498           0.176471   
               038b763d             49           0.000000   
               2eefe0ef             45           0.000000   
               0beab1cd            694           0.052632   

                         comment_lines_count  full_lines_count  \
id             cell_id                                           
00001756c60be8 1862f0a6             0.411765          0.352941   
               2a9e43d6             0.000000          0.823529   
               038b763d             0.000000          1.000000   
               2eefe0ef             0.000000          1.000000   
               0beab1cd             0.000000          0.947368   

                         text_lines_count  tag_lines_count  
id             cell_id                                      
00001756c60be8 1862f0a6               0.0              0.0  
               2a9e43d6               0.0              0.0  
               038b763d               0.0              0.0  
               2eefe0ef               0.0              0.0  
               0beab1cd               0.0              0.0

-----------------------------------------------------------------------------------------------------------------------------


100%|██████████| 100/100 [00:00<?, ?it/s]


Pandas Apply:   0%|          | 0/3 [00:00<?, ?it/s]

,id,cell_id,cell_type,source,n_code_cells,n_markdown_cells,words_count,letters_count,empty_lines_count,comment_lines_count,full_lines_count,text_lines_count,tag_lines_count,rank,ancestor_id,parent_id
0,00001756c60be8,1862f0a6,code,# This Python 3 environment comes with many h...,0.517241,0.482759,140,930,0.235294,0.411765,0.352941,0.0,0.0,0.000000,945aea18,NaN
1,00001756c60be8,2a9e43d6,code,import numpy as np\nimport pandas as pd\nimpor...,0.517241,0.482759,55,498,0.176471,0.000000,0.823529,0.0,0.0,0.034483,945aea18,NaN
2,00001756c60be8,038b763d,code,import warnings\nwarnings.filterwarnings('igno...,0.517241,0.482759,3,49,0.000000,0.000000,1.000000,0.0,0.0,0.068966,945aea18,NaN
3,00001756c60be8,2eefe0ef,code,matplotlib.rcParams.update({'font.size': 14}),0.517241,0.482759,2,45,0.000000,0.000000,1.000000,0.0,0.0,0.103448,945aea18,NaN
4,00001756c60be8,0beab1cd,code,"def evaluate_preds(train_true_values, train_pr...",0.517241,0.482759,39,694,0.052632,0.000000,0.947368,0.0,0.0,0.137931,945aea18,NaN


Duration 13.233138799667358 seconds


In [4]:
# raise

In [5]:
# for percentages 
INTREST_PERCENT = 0.95
CODE_TYPE = 'code'
MKDN_TYPE = 'markdown'
VALIDATION_RATIO = 0.2

NOT_GENERATED_COLUMNS = ['id', 'cell_id', 'source', 'cell_type', 'rank', 'ancestor_id', 'parent_id', ]
MODEL_USELESS = ['id', 'cell_id', 'cell_type', 'ancestor_id', 'parent_id', ]
GENERATED_COLUMNS_COUNT = len(df.drop(['id', 'cell_id', 'source', 'cell_type', 'rank', 'ancestor_id', 'parent_id', ], axis=1).columns)

In [6]:
# raise

In [7]:
from transformers import DistilBertModel, DistilBertTokenizer
import torch.nn.functional as F
import torch.nn as nn

import torch as t
from torch.utils.data import DataLoader, Dataset
from sklearn.model_selection import GroupShuffleSplit


from copy import deepcopy

print('__CUDA VERSION:', t.version.cuda)
print('__CUDNN VERSION:', t.backends.cudnn.version())
print(t.cuda.is_available())
print(t.cuda.empty_cache())
print(t.cuda.memory_summary(device=None, abbreviated=False))

__CUDA VERSION: 11.5
__CUDNN VERSION: 8302
True
None
|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 0            |        cudaMalloc retries: 0         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |       0 B  |       0 B  |       0 B  |       0 B  |
|       from large pool |       0 B  |       0 B  |       0 B  |       0 B  |
|       from small pool |       0 B  |       0 B  |       0 B  |       0 B  |
|---------------------------------------------------------------------------|
| Active memory         |       0 B  |       0 B  |       0 B  |       0 B  |
|       fro

In [8]:
splitter = GroupShuffleSplit(n_splits=1, test_size=VALIDATION_RATIO, random_state=RANDOM_SEED)

def extract_items(ids, data, cell_type):
    tmp = data.loc[ids, :].reset_index(drop=True)
    return tmp[tmp.cell_type == cell_type]


# Split, keeping notebooks with a common origin (ancestor_id) together
ids_train, ids_valid = next(splitter.split(df, groups=df["ancestor_id"]))
print(ids_train[:5], ids_valid[:5])

[0 1 2 3 4] [151 152 153 154 155]


In [9]:
raise

RuntimeError: No active exception to reraise

In [9]:
start_time = time.time()

# BERT_MODEL_NAME = "microsoft/codebert-base"
BERT_MODEL_NAME = 'distilbert-base-uncased'
OPTIMIZER = 'adam' # 'nadam'

MAX_LENGTH = int(df[df.cell_type == CODE_TYPE].words_count.quantile(INTREST_PERCENT)) # max length our sequences will be (e.g. how many words from a Tweet does our model see?)
BERT_OUTPUT_FEATURES = MAX_LENGTH + GENERATED_COLUMNS_COUNT


print("Words counts to cover {percent} is: {count}".format(percent=INTREST_PERCENT, count=MAX_LENGTH))
print(f'Total number of fetures output from bert: {BERT_OUTPUT_FEATURES}')

ACCUMULATION_SETPS = 3

# model run
BATCH_SIZE = 8
EPOCHS = 1
TOTAL_MAX_LEN = 256


# extract code cells for each notebook
code_df_train = extract_items(ids_train, df, CODE_TYPE)
code_df_valid = extract_items(ids_valid, df, CODE_TYPE)
# print(code_df_train[:5], code_df_valid[:5])

# build code Dataset
cd_train_ds = m.Dataset(
    code_df_train, 
    max_len=MAX_LENGTH, 
    bert_model_name=BERT_MODEL_NAME, 
    total_max_len=TOTAL_MAX_LEN, 
    drop=MODEL_USELESS
)
cd_val_ds = m.Dataset(
    code_df_valid, 
    max_len=MAX_LENGTH, 
    bert_model_name=BERT_MODEL_NAME, 
    total_max_len=TOTAL_MAX_LEN,  
    drop=MODEL_USELESS
)

# print(cd_train_ds[0], cd_val_ds[0])

# build code DataLoader
cd_train_loader = DataLoader(cd_train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=PROCESSORS_COUNT, pin_memory=False, drop_last=True)
cd_val_loader = DataLoader(cd_val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=PROCESSORS_COUNT, pin_memory=False, drop_last=False)
# print(cd_train_loader, cd_val_loader)

########################################################################################################################
code_model = m.BModel(
    BERT_MODEL_NAME, 
    # BERT_OUTPUT_FEATURES
).cuda()

code_model, code_y_pred = m.train(
    code_model, 
    cd_train_loader, 
    cd_val_loader, 
    epochs=EPOCHS, 
    accumulation_steps=ACCUMULATION_SETPS, 
    model_name=BERT_MODEL_NAME,
    opt=OPTIMIZER, 
    path=os.path.join(models_dir, 'code_bert_checkpoint.pt')
)

code_df_valid["bcpred"] = code_df_valid.groupby(["id", "cell_type"])["rank"].rank(pct=True)
code_df_valid["bcpred"] = code_y_pred

# code_gmodel = m.GModel(gc_count=GENERATED_COLUMNS_COUNT).cuda()
# code_gmodel, cd_y_gpred = m.train(code_gmodel, cd_train_loader, cd_val_loader, epochs=10, patience=3, path='pt_models/code_gen_checkpoint.pt')

# code_df_valid["gcpred"] = code_df_train.groupby(["id", "cell_type"])["rank"].rank(pct=True)
# code_df_valid["gcpred"] = cd_y_gpred

# code_df_valid['pred'] = (code_df_valid["bcpred"] + code_df_valid["gcpred"]) / 2.002
code_df_valid['pred'] = code_df_valid["bcpred"]
########################################################################################################################
code_y_dummy = code_df_valid.sort_values("pred").groupby('id')['cell_id'].apply(list)
print('Final accuracy for code is:', r.kendall_tau(df_orders.loc[code_y_dummy.index], code_y_dummy))

# best_model_state = deepcopy(code_model.state_dict())
# t.save(best_model_state, f'./pt_models/code_model_state_dict.pt')
# t.save(code_model, f'./pt_models/code_model.pt')

rduration = time.time() - start_time
print(f"\nDuration {rduration} seconds")

Words counts to cover 0.95 is: 95
Total number of fetures output from bert: 104


In [ ]:
raise

In [11]:
start_time = time.time()

BERT_MODEL_NAME = 'distilbert-base-uncased'

MAX_LENGTH = int(df[df.cell_type == MKDN_TYPE].words_count.quantile(INTREST_PERCENT)) 

# SHUFFLE_SIZE = len(df[df.cell_type == MKDN_TYPE].n_total_cells[df.n_total_cells <= SHUFFE_REF]) 
BERT_OUTPUT_FEATURES = MAX_LENGTH + GENERATED_COLUMNS_COUNT


print("Words counts to cover {percent} is: {count}".format(percent=INTREST_PERCENT, count=MAX_LENGTH))
# print("Shuffle size by {percent} count is: {count}".format(percent=SHUFFLE_INTREST_PERCENT, count=SHUFFLE_SIZE))
print(f'Total number of fetures output from bert: {BERT_OUTPUT_FEATURES}')

ACCUMULATION_SETPS = 3

# model run
BATCH_SIZE = 8
EPOCHS = 1
TOTAL_MAX_LEN = 256


# extract markdown cells for each notebook
mkdn_df_train = extract_items(ids_train, df, MKDN_TYPE)
mkdn_df_valid = extract_items(ids_valid, df, MKDN_TYPE)
# print(mkdn_df_train[:5], mkdn_df_valid[:5])


# build markdown Dataset
mkdn_train_ds = m.Dataset(
    mkdn_df_train, 
    max_len=MAX_LENGTH, 
    bert_model_name=BERT_MODEL_NAME, 
    total_max_len=TOTAL_MAX_LEN, 
    drop=MODEL_USELESS
)
mkdn_val_ds = m.Dataset(
    mkdn_df_valid, 
    max_len=MAX_LENGTH, 
    bert_model_name=BERT_MODEL_NAME, 
    total_max_len=TOTAL_MAX_LEN,  
    drop=MODEL_USELESS
)
# print(mkdn_train_ds[0], mkdn_val_ds[0])

# build markdown DataLoader
mkdn_train_loader = DataLoader(mkdn_train_ds, batch_size=BATCH_SIZE, shuffle=True, num_workers=PROCESSORS_COUNT, pin_memory=False, drop_last=True)
mkdn_val_loader = DataLoader(mkdn_val_ds, batch_size=BATCH_SIZE, shuffle=False, num_workers=PROCESSORS_COUNT, pin_memory=False, drop_last=False)
# print(mkdn_train_loader, mkdn_val_loader)

########################################################################################################################
mkdn_model = m.BModel(
    BERT_MODEL_NAME, 
    # BERT_OUTPUT_FEATURES
).cuda()




mkdn_model, mkdn_y_pred = m.train(
    mkdn_model, 
    mkdn_train_loader, 
    mkdn_val_loader, 
    epochs=EPOCHS, 
    accumulation_steps=ACCUMULATION_SETPS, 
    model_name=BERT_MODEL_NAME,
    opt='nadam', 
    path=os.path.join(models_dir, 'markdown_bert_checkpoint.pt')
)

mkdn_df_valid["bmpred"] = mkdn_df_train.groupby(["id", "cell_type"])["rank"].rank(pct=True)
mkdn_df_valid["bmpred"] = mkdn_y_pred

# mkdn_gmodel = m.GModel(gc_count=GENERATED_COLUMNS_COUNT).cuda()
# mkdn_gmodel, mkdn_y_gpred = m.train(mkdn_gmodel, mkdn_train_loader, mkdn_val_loader, epochs=10, patience=3, path='pt_models/mkdn_gen_checkpoint.pt')

# mkdn_df_valid["gmpred"] = mkdn_df_train.groupby(["id", "cell_type"])["rank"].rank(pct=True)
# mkdn_df_valid["gmpred"] = mkdn_y_gpred


# mkdn_df_valid['pred'] = (mkdn_df_valid["bmpred"] + mkdn_df_valid["gmpred"]) / 2.002
mkdn_df_valid['pred'] = mkdn_df_valid["bmpred"]
########################################################################################################################

mkdn_y_dummy = mkdn_df_valid.sort_values("pred").groupby('id')['cell_id'].apply(list)
print('Final accuracy for markdown is:', r.kendall_tau(df_orders.loc[mkdn_y_dummy.index], mkdn_y_dummy))

# # best_model_state = deepcopy(mkdn_model.state_dict())
# # t.save(best_model_state, f'./pt_models/markdown_model_state_dict.pt')
# # t.save(mkdn_model, f'./pt_models/markdown_model.pt')

rduration = time.time() - start_time
print(f"\nDuration {rduration} seconds")

Words counts to cover 0.95 is: 116
Total number of fetures output from bert: 125


In [ ]:
raise

In [19]:
CODE_TYPE = 'code'
MKDN_TYPE = 'markdown'

BERT_MODEL_NAME = 'distilbert-base-uncased'
PROCESSORS_COUNT = psutil.cpu_count(logical=False)

BATCH_SIZE = 8
TOTAL_MAX_LEN = 256

code_df_valid['rank'] = m.predict(
    model_path=BERT_MODEL_NAME,
    check_point=os.path.join(models_dir, 'code_bert_checkpoint.pt'), 
    batch_size=BATCH_SIZE, 
    num_workers=PROCESSORS_COUNT, 
    max_len=95, 
    total_max_len=TOTAL_MAX_LEN, 
    data=code_df_valid, 
    drop=MODEL_USELESS
)

mkdn_df_valid['rank'] = m.predict(
    model_path=BERT_MODEL_NAME,
    check_point=os.path.join(models_dir, 'markdown_bert_checkpoint.pt'), 
    batch_size=BATCH_SIZE, 
    num_workers=PROCESSORS_COUNT, 
    max_len=116, 
    total_max_len=TOTAL_MAX_LEN, 
    data=mkdn_df_valid, 
    drop=MODEL_USELESS
)

vres_df = pd.concat([mkdn_df_valid, code_df_valid], ignore_index=True, ).sort_values("rank").groupby("id")["cell_id"].apply(list)
print('Final total accuracy is:', r.kendall_tau(df_orders.loc[vres_df.index], vres_df))

display(vres_df.head())

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


100%|██████████| 96/96 [00:29<00:00,  3.22it/s]


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.weight', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight', 'vocab_projector.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


100%|██████████| 49/49 [00:29<00:00,  1.68it/s]
Final total accuracy is: 0.08832626051931236


id
0001bdd4021779    [3fdc37be, 38310c80, 7fde4f04, ad7679ef, 0a1a7...
0007f21ee357b5    [40a1a0ce, e68e6c44, 7a5263a6, 28660f19, 4d264...
000890decea38e    [ec7e340c, d1b548da, b444859f, 3d243130, d46fa...
0008ba887b3817    [006235ba, 12f6aba1, 3231cc0c, 5bd9ae73, bf698...
000af347c31e72    [6fa470b3, 6c44e9ba, a01ba7ea, 98d1cdf3, 00c42...
Name: cell_id, dtype: object

In [ ]:
raise

In [20]:
test_dir = os.path.join(data_dir, 'test')
print(f"\033[94mNumber of notebooks present in test set  = ", len(list(glob.iglob(os.path.join(test_dir, '*.json')))))

df_test = r.read_all_notebooks_(test_dir, 4, 2, desc="Tests NBs")

df_test = r.extract_features(df_test).reset_index()
df_test['rank'] = 0

MODEL_USELESS = ['id', 'cell_id', 'cell_type', ]

CODE_TYPE = 'code'
MKDN_TYPE = 'markdown'

BERT_MODEL_NAME = 'distilbert-base-uncased'
PROCESSORS_COUNT = psutil.cpu_count(logical=False)

BATCH_SIZE = 8
TOTAL_MAX_LEN = 256

df_test.loc[df_test.cell_type == CODE_TYPE, 'rank'] = m.predict(
    model_path=BERT_MODEL_NAME,
    check_point=os.path.join(models_dir, 'code_bert_checkpoint.pt'), 
    batch_size=BATCH_SIZE, 
    num_workers=PROCESSORS_COUNT, 
    max_len=95, 
    total_max_len=TOTAL_MAX_LEN, 
    data=df_test[df_test.cell_type == CODE_TYPE], 
    drop=MODEL_USELESS
)

df_test.loc[df_test.cell_type == MKDN_TYPE, 'rank'] = m.predict(
    model_path=BERT_MODEL_NAME,
    check_point=os.path.join(models_dir, 'markdown_bert_checkpoint.pt'), 
    batch_size=BATCH_SIZE, 
    num_workers=PROCESSORS_COUNT, 
    max_len=116, 
    total_max_len=TOTAL_MAX_LEN, 
    data=df_test[df_test.cell_type == MKDN_TYPE], 
    drop=MODEL_USELESS
)

df_test = df_test.sort_values("rank").groupby("id")["cell_id"].apply(lambda x: " ".join(x)).reset_index()
df_test.rename(columns={"cell_id": "cell_order"}, inplace=True)
display(df_test.head())

df_test.to_csv("submission.csv", index=False)

Number of notebooks present in test set  =  4


Tests NBs: 100%|██████████| 4/4 [00:00<00:00, 4007.94it/s]


<class 'pandas.core.frame.DataFrame'>
MultiIndex: 89 entries, ('0009d135ece78d', 'ddfd239c') to ('0028856e09c5b7', 'eb293dfc')
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype   
---  ------     --------------  -----   
 0   cell_type  89 non-null     category
 1   source     89 non-null     object  
dtypes: category(1), object(1)
memory usage: 4.1+ KB
None
Extract Code Cells Counts
Extract Markdown Cells Counts


Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


100%|██████████| 6/6 [00:29<00:00,  4.98s/it]


C:\Users\yazee\AppData\Local\Temp\ipykernel_14212\1402259125.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[df_test.cell_type == CODE_TYPE]['rank'] = m.predict(
Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_projector.weight', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_layer_norm.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoin

100%|██████████| 6/6 [00:29<00:00,  4.86s/it]


C:\Users\yazee\AppData\Local\Temp\ipykernel_14212\1402259125.py:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_test[df_test.cell_type == MKDN_TYPE]['rank'] = m.predict(
